In [95]:
import pandas as pd

In [96]:
df = pd.read_csv('../data/raw/data.csv', low_memory=False)
df.head()

,ID,Locality,Postal Code,Build Year,Facades,Habitable Surface,Land Surface,Type,Subtype,Price,...,Is Holiday Property,Gas Water Electricity,Sewer,Sea view,Parking count inside,Parking count outside,url,Latitude,Longitude,Consumption
0,11129943,Heusden-Zolder,3550,NaN,4.0,394.0,10000.0,HOUSE,HOUSE,1399000,...,NaN,False,CONNECTED,NaN,NaN,NaN,https://www.immoweb.be/en/classified/house/for...,NaN,NaN,115.0
1,11188967,Uccle,1180,2013.0,4.0,840.0,4425.0,HOUSE,HOUSE,4950000,...,NaN,NaN,NaN,NaN,2.0,2.0,https://www.immoweb.be/en/classified/house/for...,NaN,NaN,61.0
2,9741047,Ath,7800,NaN,NaN,170.0,583.0,HOUSE,HOUSE,372054,...,False,False,NOT_CONNECTED,NaN,NaN,NaN,https://www.immoweb.be/en/classified/house/for...,50.633501,3.788273,NaN
3,10988804,Enghien,7850,NaN,3.0,1104.0,1840.0,HOUSE,APARTMENT_BLOCK,1800000,...,NaN,NaN,NaN,NaN,2.0,3.0,https://www.immoweb.be/en/classified/house/for...,NaN,NaN,832.0
4,10551894,Uccle,1180,1920.0,4.0,650.0,2000.0,HOUSE,VILLA,3950000,...,NaN,NaN,NaN,NaN,2.0,4.0,https://www.immoweb.be/en/classified/house/for...,NaN,NaN,NaN


In [97]:
# print all the columns for auto-completion
columns = ['ID', 'Locality', 'Postal Code', 'Build Year', 'Facades',
       'Habitable Surface', 'Land Surface', 'Type', 'Subtype', 'Price',
       'Sale Type', 'Bedroom Count', 'Bathroom Count', 'Toilet Count',
       'Room Count', 'Kitchen Surface', 'Kitchen', 'Kitchen Type', 'Furnished',
       'Openfire', 'Fireplace Count', 'Terrace', 'Terrace Surface',
       'Terrace Orientation', 'Garden Exists', 'Garden Surface',
       'Garden Orientation', 'Swimming Pool', 'State of Building',
       'Living Surface', 'EPC', 'Cadastral Income', 'Has starting Price',
       'Transaction Subtype', 'Heating Type', 'Is Holiday Property',
       'Gas Water Electricity', 'Sewer', 'Sea view', 'Parking count inside',
       'Parking count outside', 'url', 'Latitude', 'Longitude']
# df.columns

# split the data

In [98]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Price'])
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=41, test_size=0.2)

train_data = pd.concat([X_train, y_train], axis=1)

In [99]:
# calculate a dataframe with the correlation to the price for all numeric features

numeric_training_data = train_data.select_dtypes(include=['int64', 'float64'])

correlation = numeric_training_data.corr()['Price'].sort_values(ascending=False)
correlation = correlation.drop('Price')
correlation = correlation.to_frame()
correlation

,Price
Bathroom Count,0.351623
Bedroom Count,0.345054
Latitude,0.225603
Terrace Surface,0.224730
Fireplace Count,0.213150
Habitable Surface,0.201494
Garden Surface,0.161538
Land Surface,0.158689
Facades,0.148851
Living Surface,0.038944


In [100]:
keep_columns = ['Bathroom Count', 'Bedroom Count', 'Habitable Surface', 'Land Surface', 'Facades', 'Subtype']
df = df[keep_columns + ['Price']]

In [101]:
keep_subtypes = ['VILLA', 'HOUSE', 'APARTMENT', 'MANSION', 'PENTHOUSE', 'TOWN_HOUSE', 'GROUND_FLOOR', 'FLAT_STUDIO', 'DUPLEX',]

In [102]:
df = df[df['Subtype'].isin(keep_subtypes)]

In [103]:
from sklearn.preprocessing import OneHotEncoder

# one hot encode the subtypes as numeric values 0 or 1
ohe_subtypes = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
ohe_subtypes.fit(df[['Subtype']])
df_ohe = ohe_subtypes.transform(df[['Subtype']])
df = pd.concat([df, df_ohe], axis=1)
df.drop('Subtype', axis=1, inplace=True)
df

,Bathroom Count,Bedroom Count,Habitable Surface,Land Surface,Facades,Price,Subtype_APARTMENT,Subtype_DUPLEX,Subtype_FLAT_STUDIO,Subtype_GROUND_FLOOR,Subtype_HOUSE,Subtype_MANSION,Subtype_PENTHOUSE,Subtype_TOWN_HOUSE,Subtype_VILLA
0,2.0,3.0,394.0,10000.0,4.0,1399000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,5.0,6.0,840.0,4425.0,4.0,4950000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,NaN,3.0,170.0,583.0,NaN,372054,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5.0,6.0,650.0,2000.0,4.0,3950000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,5.0,5.0,530.0,213.0,2.0,3950000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88854,1.0,4.0,180.0,NaN,4.0,150000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88855,1.0,4.0,180.0,NaN,4.0,150000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88856,2.0,3.0,NaN,NaN,2.0,150000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88857,1.0,2.0,89.0,NaN,2.0,260000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
# Give me a df that shows the correlation with the price 

correlation = df.corr()['Price'].sort_values(ascending=False)
correlation = correlation.drop('Price')
correlation = correlation.to_frame()
correlation

,Price
Bathroom Count,0.321298
Bedroom Count,0.316904
Subtype_VILLA,0.266714
Land Surface,0.241080
Habitable Surface,0.162396
Facades,0.158486
Subtype_PENTHOUSE,0.084503
Subtype_MANSION,0.072895
Subtype_DUPLEX,0.016861
Subtype_TOWN_HOUSE,-0.010920


In [105]:
# Drop all subtype columns except for Subtype_VILLA
df.drop(['Subtype_APARTMENT', 'Subtype_DUPLEX', 'Subtype_FLAT_STUDIO', 'Subtype_GROUND_FLOOR', 'Subtype_MANSION', 'Subtype_PENTHOUSE', 'Subtype_TOWN_HOUSE'], axis=1, inplace=True)
df

,Bathroom Count,Bedroom Count,Habitable Surface,Land Surface,Facades,Price,Subtype_HOUSE,Subtype_VILLA
0,2.0,3.0,394.0,10000.0,4.0,1399000,1.0,0.0
1,5.0,6.0,840.0,4425.0,4.0,4950000,1.0,0.0
2,NaN,3.0,170.0,583.0,NaN,372054,1.0,0.0
4,5.0,6.0,650.0,2000.0,4.0,3950000,0.0,1.0
5,5.0,5.0,530.0,213.0,2.0,3950000,0.0,0.0
...,...,...,...,...,...,...,...,...
88854,1.0,4.0,180.0,NaN,4.0,150000,0.0,0.0
88855,1.0,4.0,180.0,NaN,4.0,150000,0.0,0.0
88856,2.0,3.0,NaN,NaN,2.0,150000,0.0,0.0
88857,1.0,2.0,89.0,NaN,2.0,260000,0.0,0.0


In [106]:
# give me a df that shows the percentage of missing values for each column
total_rows = df.shape[0]
missing_values = df.isnull().mean().sort_values(ascending=False)
missing_values = missing_values[missing_values > 0]
missing_values = missing_values.to_frame()

In [107]:
from sklearn.preprocessing import MinMaxScaler
from features.generic_transformer import MyKnnImputer, MyMinMaxScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
       ("MinMaxScaler", MyMinMaxScaler({"Price": 3})),
       ("Impute Land surface", MyKnnImputer(column_names=['Habitable Surface', 'Land Surface', 'Facades', 'Subtype_APARTMENT'], n_neighbors=5)),
])

df = pipeline.fit_transform(df)
total_rows = df.shape[0]
missing_values = df.isnull().mean().sort_values(ascending=False)
missing_values = missing_values[missing_values > 0]
missing_values = missing_values.to_frame()

AttributeError: 'numpy.ndarray' object has no attribute 'columns'